In [1]:
import pandas as pd
import numpy as np
from ggv.utils.context import create_default_context
import warnings
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')
context = create_default_context()
bq_db= context.get_ds('ANALYTICS_BIG_QUERY')

Creating context with config from: /home/jovyan/docker/config.yml
Setting up logging with /home/jovyan/docker/log.yml
INFO - ggv.utils.vault - Loading secret from vault: gogotech/data/data_team/databases/ANALYTICS_BIG_QUERY
INFO - ggv.utils.vault - Creating new connection to vault https://vault-v2.gogo.tech
INFO - ggv.utils.data.sources.bigquery_ds - Using dataset: gogox-data-science-non-prod.raw


In [2]:
query= f""" 
SELECT driver_id, 
       vehicle, 
       vehicle_year,
       vehicle_insurance_expiry_date,  
       CASE 
       WHEN vehicle_cd= 0 THEN 'VAN'
       WHEN vehicle_cd= 1 THEN 'MUDOU'
       WHEN vehicle_cd= 3 THEN 'MUDOU9'
       WHEN vehicle_cd= 10 THEN 'MOTORCYCLE'
       END AS vehicle_type
FROM `gogox-data-science-non-prod.analytics_prod_master.driver_master_table_hk` 
WHERE is_suspended= False 
AND transport_order_cap != 0 
AND verification_status = 'finished'
AND vehicle_cd != 7
AND vehicle_cd !=2  
"""
driver_vehicle= bq_db.df_from_sql(query)

In [38]:
driver_vehicle['vehicle_insurance_expiry_date']= pd.to_datetime(driver_vehicle['vehicle_insurance_expiry_date'])
driver_vehicle['vehicle_year']= driver_vehicle['vehicle_year'].fillna(0).astype(int)
driver_vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10105 entries, 0 to 10104
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   driver_id                      10105 non-null  int64         
 1   vehicle                        9767 non-null   object        
 2   vehicle_year                   10105 non-null  int64         
 3   vehicle_insurance_expiry_date  9702 non-null   datetime64[ns]
 4   vehicle_type                   10105 non-null  object        
 5   insurance_expiry_month_year    10105 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 473.8+ KB


In [39]:
#get month and year from vehicle_insurance_expiry_date and combine them 
driver_vehicle['insurance_expiry_month_year']= (driver_vehicle['vehicle_insurance_expiry_date'].dt.month.fillna(0).astype(int).astype(str)+ 
                                                '-'+ 
                                                driver_vehicle['vehicle_insurance_expiry_date'].dt.year.fillna(0).astype(int).astype(str)) 

driver_vehicle                                            

,driver_id,vehicle,vehicle_year,vehicle_insurance_expiry_date,vehicle_type,insurance_expiry_month_year
0,40204,None,0,NaT,MOTORCYCLE,0-0
1,41118,None,0,NaT,MOTORCYCLE,0-0
2,42665,None,0,NaT,MOTORCYCLE,0-0
3,41016,None,0,NaT,MOTORCYCLE,0-0
4,42375,None,0,2020-05-22,MOTORCYCLE,5-2020
...,...,...,...,...,...,...
10100,42267,h1,2018,2022-10-23,VAN,10-2022
10101,68619,vito,2011,2023-04-08,VAN,4-2023
10102,42595,h1,2018,2021-11-08,VAN,11-2021
10103,63531,hiace,2012,2022-02-21,VAN,2-2022


In [40]:
driver_vehicle

,driver_id,vehicle,vehicle_year,vehicle_insurance_expiry_date,vehicle_type,insurance_expiry_month_year
0,40204,None,0,NaT,MOTORCYCLE,0-0
1,41118,None,0,NaT,MOTORCYCLE,0-0
2,42665,None,0,NaT,MOTORCYCLE,0-0
3,41016,None,0,NaT,MOTORCYCLE,0-0
4,42375,None,0,2020-05-22,MOTORCYCLE,5-2020
...,...,...,...,...,...,...
10100,42267,h1,2018,2022-10-23,VAN,10-2022
10101,68619,vito,2011,2023-04-08,VAN,4-2023
10102,42595,h1,2018,2021-11-08,VAN,11-2021
10103,63531,hiace,2012,2022-02-21,VAN,2-2022


### Vehicle's Year of manufacture

In [41]:
driver_vehicle.loc[driver_vehicle['vehicle_year']< 1980, 'vehicle_year']=0 
driver_vehicle.loc[driver_vehicle['vehicle_year']> 2022, 'vehicle_year']=0 

In [42]:
driver_vehicle.vehicle_year.unique()

array([   0, 2018, 2017, 2012, 2015, 2013, 2004, 2014, 2006, 2019, 2016,
       2021, 2011, 2010, 2020, 2022, 2009, 2007, 2008, 1990, 2005, 1999,
       1997, 2001, 2003, 2002, 1998, 2000])

In [43]:
manufacture_year= driver_vehicle.loc[driver_vehicle['vehicle_year']!=0]

year= [str(i) for i in sorted(manufacture_year['vehicle_year'].unique())]

driver_count= [manufacture_year[manufacture_year['vehicle_year']==
                                         int(i)]['driver_id'].count() for i in year] 


driver_perc = [round(i/sum(driver_count)*100, 4) for i in driver_count]


In [44]:
year_distribution= pd.DataFrame({'manufacture_year': year,
                                 'driver_count': driver_count, 
                                'percentage': driver_perc})

df=year_distribution.sort_values(by='percentage', ascending=False).reset_index(drop=True)
df.head()

,manufacture_year,driver_count,percentage
0,2015,1057,10.9149
1,2014,1053,10.8736
2,2017,921,9.5105
3,2018,881,9.0975
4,2016,749,7.7344


In [81]:
fig= go.Figure(go.Bar(x=year_distribution['manufacture_year'], 
                      y=year_distribution['driver_count'], 
                      marker_color= '#00CC96'))
fig.update_layout(title='Distribution of manufacture year', 
                 xaxis_title= 'no. of drivers', 
                 yaxis_title='vehicle manufacture year')
fig.show() 

remark: 
<br>The reliability of the data is uncertain, as there is much data don't make sense, like manfacture year = 1230 

## Car model

In [46]:
driver_vehicle.vehicle.unique()

array([None, 'expert', 'hiace', 'vito', 'transporter', 'h1', 'others',
       'transit', 'urvan', 'kangoo', 'traffic'], dtype=object)

In [47]:
null_vehicle_driver= driver_vehicle[driver_vehicle['vehicle'].isnull()]['driver_id'].count() 

null_vehicle_perc= round((driver_vehicle[driver_vehicle['vehicle'].isnull()]['driver_id'].count()
                          / driver_vehicle.driver_id.count()), 2)

print(f'Percentage of drivers with null car model: {null_vehicle_perc}')

Percentage of drivers with null car model: 0.03


In [48]:
driver_vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10105 entries, 0 to 10104
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   driver_id                      10105 non-null  int64         
 1   vehicle                        9767 non-null   object        
 2   vehicle_year                   10105 non-null  int64         
 3   vehicle_insurance_expiry_date  9702 non-null   datetime64[ns]
 4   vehicle_type                   10105 non-null  object        
 5   insurance_expiry_month_year    10105 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 473.8+ KB


In [49]:
model= [i for i in driver_vehicle.vehicle.unique()]
model.pop(0)

model_count= [driver_vehicle[driver_vehicle['vehicle']==i]['driver_id'].count() for i in model]
model_count.insert(0, driver_vehicle[driver_vehicle['vehicle'].isnull()]['driver_id'].count())

model_perc= [round(i / driver_vehicle.driver_id.count(), 4)*100 for i in model_count]

model.insert(0, 'null')

In [50]:
car_model= pd.DataFrame({'model': model, 
                        'driver_count': model_count, 
                        'driver_percent': model_perc})

car_model.sort_values(by='driver_percent', ascending=False).reset_index(drop=True)

,model,driver_count,driver_percent
0,hiace,4555,45.08
1,vito,1467,14.52
2,h1,1347,13.33
3,others,1280,12.67
4,transporter,368,3.64
5,urvan,346,3.42
6,null,338,3.34
7,expert,218,2.16
8,transit,184,1.82
9,kangoo,1,0.01


In [51]:
#find vehicle_type for those vehicle is null / vehicle = 'others'
null_other_model= driver_vehicle.loc[(driver_vehicle['vehicle'].isna()) | 
                                    (driver_vehicle['vehicle']=='others')]
null_other_model.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1618 entries, 0 to 10099
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   driver_id                      1618 non-null   int64         
 1   vehicle                        1280 non-null   object        
 2   vehicle_year                   1618 non-null   int64         
 3   vehicle_insurance_expiry_date  1303 non-null   datetime64[ns]
 4   vehicle_type                   1618 non-null   object        
 5   insurance_expiry_month_year    1618 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 88.5+ KB


among null values and 'others', the distribution of vehicle_type is as below: 

In [52]:
vehicle_null_other= [i for i in null_other_model.vehicle_type.unique()]

vehicle_null_other_count= [null_other_model[null_other_model['vehicle_type']==i]['driver_id'].count() 
                           for i in vehicle_null_other]

vehicle_null_other_perc= [round(i / null_other_model.driver_id.count(), 4)*100 
                          for i in vehicle_null_other_count]


df_null_other= pd.DataFrame({'vehicle_type': vehicle_null_other,
                             'driver_count': vehicle_null_other_count, 
                             'driver_percent': vehicle_null_other_perc})

df_null_other.sort_values(by='driver_percent', ascending=False).reset_index(drop=True)

,vehicle_type,driver_count,driver_percent
0,MUDOU9,703,43.45
1,MUDOU,527,32.57
2,MOTORCYCLE,298,18.42
3,VAN,90,5.56


### Vehicle Type

In [53]:
vtype= [str(i) for i in driver_vehicle.vehicle_type.unique()] 

type_count = [driver_vehicle[driver_vehicle['vehicle_type']==i]['driver_id'].count() for i in vtype]

In [80]:
fig = go.Figure(data=[go.Pie(labels=vtype, values=type_count)])
fig.update_layout(title='Distribution of Vehicle Types')
fig.show()

### Insurance expiry date 

In [57]:
driver_vehicle.loc[driver_vehicle['insurance_expiry_month_year']=='0-0', 'insurance_expiry_month_year'] = '0' 

In [58]:
driver_vehicle.insurance_expiry_month_year.unique() 

array(['0', '5-2020', '6-2021', '4-2019', '7-2021', '8-2022', '5-2018',
       '6-2018', '6-2020', '8-2018', '8-2020', '5-2022', '11-2021',
       '9-2022', '3-2022', '12-2021', '1-2025', '1-2022', '7-2022',
       '9-2021', '11-2022', '2-2022', '1-2021', '5-2023', '6-2023',
       '6-2022', '4-2022', '12-2022', '3-2020', '4-2023', '3-2023',
       '8-2021', '10-2021', '2-2042', '10-2022', '2-2023', '7-2023',
       '4-2021', '6-2019', '9-2023', '1-2023', '8-2023', '12-2016',
       '9-2018', '5-2019', '11-2027', '10-2023', '11-2020', '12-2023',
       '2-2019', '11-2023', '2-2021', '9-2017', '1-2024', '7-2019',
       '3-2024', '5-2021', '12-2020', '3-2021', '7-2020', '5-2026',
       '3-2019', '1-2020', '4-2020', '2-2020', '9-2020', '11-2018',
       '11-2019', '12-2019', '9-2019', '3-2018', '10-2020', '7-2018',
       '10-2019', '8-2019', '12-2024', '11-2017', '2-2018', '12-2018',
       '1-2019', '9-2039', '10-2018', '10-2017', '4-2018', '2-2026',
       '7-2017', '5-2017', '9-2016

In [64]:
driver_vehicle= driver_vehicle.sort_values(by='vehicle_insurance_expiry_date').reset_index(drop=True)

In [69]:
expiry_month_year= [i for i in driver_vehicle.insurance_expiry_month_year.unique()]
#drop 0 from list
expiry_month_year.pop(-1)

driver_ct= [driver_vehicle[driver_vehicle['insurance_expiry_month_year']==i]['driver_id'].count() 
            for i in expiry_month_year]

FYI since there is no relevant data for insurance start month, insurnace expiry month is provided in stead

In [79]:
fig= go.Figure(go.Bar(x=expiry_month_year, 
                      y=driver_ct, 
                      marker_color= '#AB63FA'))

fig.update_xaxes(tickangle=50)
fig.update_layout(title='Distribution of insurance expiry date', 
                 xaxis_title='insurance expiry month', 
                 yaxis_title='no. of drivers')
fig.show() 

### Avg online time per day

In [ ]:
query= f""" 
SELECT date, driver_id, MOD(time_slot, 1 << 12) as online_mins 
FROM `gogox-analytics-prod.hk_db.driver_onlines`
CROSS JOIN UNNEST(time_slots) as time_slot 
WHERE date >= DATE_SUB(CURRENT_DATETIME(), INTERVAL '1' YEAR)
ORDER BY date 
"""  
online_mins= bq_db.df_from_sql(query)

In [ ]:
total_mins = online_mins.groupby(['date', 'driver_id']).agg({'online_mins':'sum'})
total_mins = total_mins.reset_index(level=0).reset_index(level=0)
total_mins

In [ ]:
daily_online_mins= total_mins.groupby('driver_id').agg({'date': 'count', 
                                                            'online_mins':'mean'}).reset_index(
                                                                                        level=0)
daily_online_mins= daily_online_mins.rename(columns={'date':'online_date_count'})
daily_online_mins

In [69]:
#avg online time per day of all GGX drivers 
print('avg online per day of GGX driver:', round(daily_online_mins.online_mins.mean(),2), 'mins')

avg online per day of GGX driver: 52.56 mins


In [35]:
#get the names for each bar 
x=[] 
for i in range(1,10): 
    x.append(str(i)) 
x.insert(0, '0')
x.insert(1, '<1')
x.extend(['10-20','20-30','30-40','40-50','50-60','1 hr', '2 hrs', '>2 hrs'])

In [36]:
#get number of drivers in different online_min 
y=[] 
y.insert(0, daily_online_mins[daily_online_mins['online_mins']==0]['driver_id'].count())
y.insert(1, daily_online_mins[(daily_online_mins['online_mins']>0) & 
                                               (daily_online_mins['online_mins']<1)]['driver_id'].count())

for i in range(1,10): 
    j= i+1 
    y.append(daily_online_mins[(daily_online_mins['online_mins']>=i) & 
                                               (daily_online_mins['online_mins']<j)]['driver_id'].count())
    
        
for i in range(10, 60, 10): 
    j= i+10 
    y.append(daily_online_mins[(daily_online_mins['online_mins']>=i) & 
                                               (daily_online_mins['online_mins']<j)]['driver_id'].count())

y.insert(16, daily_online_mins[(daily_online_mins['online_mins']>=60) & 
                                               (daily_online_mins['online_mins']< 120)]['driver_id'].count())    

y.insert(17, daily_online_mins[(daily_online_mins['online_mins']>=120) & 
                                               (daily_online_mins['online_mins']< 240)]['driver_id'].count())

y.insert(18, daily_online_mins[daily_online_mins['online_mins']>= 240]['driver_id'].count())

print(y)

[1382, 2329, 2670, 1925, 1525, 1112, 912, 738, 636, 561, 489, 2952, 1711, 1220, 983, 818, 3102, 2971, 1541]


In [37]:
#data based on last 1 year 
fig= go.Figure(go.Bar(x=x,y=y)) 
fig.update_layout(title='Distribution of Average Daily Online Time (in minutes)',
                  xaxis_title='Time (in minutes)',yaxis_title='Number of Drivers')
fig.show()

Note that some drivers will only be online a few days in a year

In [38]:
#seems like drivers who online more will be online longer 
daily_online_mins[daily_online_mins['online_date_count']>= 100]['online_mins'].mean()

126.44294643782018

In [39]:
daily_online_mins[daily_online_mins['online_date_count']< 100]['online_mins'].mean()

18.58614488717974